In [1]:
import numpy as np
import scipy.io 
import scipy.misc
import tensorflow as tf 
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
output_dir = 'outputs/'
style_dir = 'styles/'
content_dir = 'contents/'
# image dimensions 
img_width = 800
img_height = 600
channels = 3

In [5]:
# Noise ratio. Percentage of weight of the noise for intermixing with the
# content image.
noise = 0.6
# parameter to put more emphasis on content loss.
beta = 5
# parameter to put more emphasis on style loss.
alpha = 100

vgg_path = 'imagenet-vgg-verydeep-19.mat'
# vgg model was downloaded from this site
# http://www.vlfeat.org/matconvnet/pretrained/#downloading-the-pre-trained-models

# # The mean to subtract from the input to the VGG model. This is the mean that
# # when the VGG was used to train. Minor changes to this will make a lot of
# # difference to the performance of model.
# MEAN_VALUES = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))

In [102]:
def get_weights(vgg, layer):
    W = vgg['layers'][0][layer][0][0][2][0][0] 
    b = vgg['layers'][0][layer][0][0][2][0][1]
    return W, b

def conv2d(vgg, prev_layer, layer):
    W, b = get_weights(vgg, layer)
    W = tf.constant(W)
    b = tf.constant(np.reshape(b, (b.shape[0],)))
    conv2d_layer = tf.nn.conv2d(prev_layer, W, strides=[1, 1, 1, 1], padding='SAME') + b
    return conv2d_layer

def relu(conv2d_layer):
    return tf.nn.relu(conv2d_layer)

def avg_pool(prev_layer):
    return tf.nn.avg_pool(prev_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def load_vgg(model_path):
    model = {}
    vgg = scipy.io.loadmat(model_path)
    model["input"] = tf.Variable(np.zeros((1, img_height, img_width, channels), dtype=np.float32))

    # group 1
    model["conv1_1"] = conv2d(vgg, model["input"], 0)
    model["relu1_1"] = relu(model["conv1_1"])
    model["conv1_2"] = conv2d(vgg, model["relu1_1"], 2)
    model["relu1_2"] = relu(model["conv1_2"])
    model["avg_pool1"] = avg_pool(model["relu1_2"])

    # group 2
    model["conv2_1"] = conv2d(vgg, model["avg_pool1"], 5)
    model["relu2_1"] = relu(model["conv2_1"])
    model["conv2_2"] = conv2d(vgg, model["relu2_1"], 7)
    model["relu2_2"] = relu(model["conv2_2"])
    model["avg_pool2"] = avg_pool(model["relu2_2"])

    # group 3
    model["conv3_1"] = conv2d(vgg, model["avg_pool2"], 10)
    model["relu3_1"] = relu(model["conv3_1"])
    model["conv3_2"] = conv2d(vgg, model["relu3_1"], 12)
    model["relu3_2"] = relu(model["conv3_2"])
    model["conv3_3"] = conv2d(vgg, model["relu3_2"], 14)
    model["relu3_3"] = relu(model["conv3_3"])
    model["conv3_4"] = conv2d(vgg, model["relu3_3"], 16)
    model["relu3_4"] = relu(model["conv3_4"])
    model["avg_pool3"] = avg_pool(model["relu3_4"])

    # group 4
    model["conv4_1"] = conv2d(vgg, model["avg_pool3"], 19)
    model["relu4_1"] = relu(model["conv4_1"])
    model["conv4_2"] = conv2d(vgg, model["relu4_1"], 21)
    model["relu4_2"] = relu(model["conv4_2"])
    model["conv4_3"] = conv2d(vgg, model["relu4_2"], 23)
    model["relu4_3"] = relu(model["conv4_3"])
    model["conv4_4"] = conv2d(vgg, model["relu4_3"], 25)
    model["relu4_4"] = relu(model["conv4_4"])
    model["avg_pool4"] = avg_pool(model["relu4_4"])

    # group 5
    model["conv5_1"] = conv2d(vgg, model["avg_pool4"], 28)
    model["relu5_1"] = relu(model["conv5_1"])
    model["conv5_2"] = conv2d(vgg, model["relu5_1"], 30)
    model["relu5_2"] = relu(model["conv5_2"])
    model["conv5_3"] = conv2d(vgg, model["relu5_2"], 32)
    model["relu5_3"] = relu(model["conv5_3"])
    model["conv5_4"] = conv2d(vgg, model["relu5_3"], 34)
    model["relu5_4"] = relu(model["conv5_4"])
    model["avg_pool5"] = avg_pool(model["relu5_4"])

    return model

In [103]:
def calc_content_loss(p, x):
    M = p.shape[1] * p.shape[2] # height * width
    N = p.shape[3] # # of filters
    loss = 0.5 * tf.reduce_sum(tf.pow(x - p, 2))
    return loss

def gram_matrix(x, M, N):
    F = tf.reshape(x, (M, N))
    G = tf.matmul(tf.transpose(F), F)
    return G

def calc_style_loss(a, x):
    M = a.shape[1] * a.shape[2] # height * width
    N = a.shape[3] # # of filters
    A = gram_matrix(a, M, N) # gram matrix of original image
    G = gram_matrix(x, M, N)  # gram matrix of generated image
    loss = (1 / (4 * (N ^ 2) * (M ^ 2))) * tf.reduce_sum(tf.pow((G - A), 2))
    return loss

In [104]:
def add_noise(content_img, noise_ratio=0.5):
    """
    Add noise to the content image
    """
    noise_img = np.random.uniform(-20, 20, (1, img_height, img_width, channels)).astype('float32')
    # white noise image  
    input_img = noise_img*noise_ratio + content_img*(1 - noise_ratio)
    return input_img

In [105]:
def load_image(img_dir, img_name):
    image = scipy.misc.imread(img_dir+img_name)
    image = scipy.misc.imresize(image, (img_height, img_width))
    image = image[np.newaxis, :, :, :]
    return image

def save_image(output_dir, img_name, image):
    scipy.misc.imsave(output_dir+img_name, image[0])

In [106]:
content_image = load_image(content_dir, "Beauty.jpg")
style_image = load_image(style_dir, "Starry_night.jpg")
vgg_model = load_vgg(vgg_path)

/Users/s_wt/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/Users/s_wt/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [107]:
%%time
#need to rewrite as a func
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(vgg_model["input"].assign(content_image))
    content_loss = calc_content_loss(sess.run(vgg_model["conv4_2"]), vgg_model["conv4_2"])
    style_loss = 0
    layers = [("conv1_1", 1), ("conv2_1", 2), ("conv3_1", 3), ("conv4_1", 4), ("conv5_1", 5)]
    sess.run(vgg_model["input"].assign(style_image))
    for layer in layers:
        E = calc_style_loss(sess.run(vgg_model[layer[0]]), vgg_model[layer[0]])
        W = layer[1]
        style_loss += E * W
    total_loss = (alpha * content_loss) + (beta * style_loss)
    iterations = 1000
    train_step = tf.contrib.opt.ScipyOptimizerInterface(total_loss, method="L-BFGS-B", 
                                                        options={"maxiter": iterations, "disp": 100})

    sess.run(tf.global_variables_initializer())
    sess.run(vgg_model["input"].assign(content_image))
    train_step.minimize(sess)

    # save the final image
    output_image = sess.run(vgg_model["input"])
    save_image(output_dir, "output0.jpg", output_image)

KeyboardInterrupt: 